In [1]:
from dataclasses import dataclass
from file_db.file_db_manager import FileDBManager

@dataclass
class MyRow:
    key: str
    name: str
    age: int

dbm = FileDBManager(base_dir="./")

db_name = "my_database"
tb_name = "my_table"
tb_new_name = "my_new_table"

In [2]:
# DB 생성 테스트.
dbm.show_dbs()
dbm.create_db(name=db_name)

try:
    dbm.create_db(name=db_name)
except Exception as e:
    print(e)

dbm.show_dbs()

- No database exists.
Database 'my_database' already exists.
- DB list: my_database


In [3]:
# 생성된 DB 조회.
db = dbm.load_db(name=db_name)

# 테이블 생성 및 명명 테스트.
db.show_tables()
db.create_table(name=tb_name, row_class=MyRow)
db.show_tables()
db.rename_table(old_name=tb_name, new_name=tb_new_name)
db.show_tables()

- No table exists.
Table 'my_table' created successfully.
- Table list: my_table
Table 'my_table' renamed to 'my_new_table'.
- Table list: my_new_table


In [4]:
# 생성된 테이블 조회.
tb = db.load_table(name=tb_new_name)

# 데이터 삽입 및 조회 테스트.
tb.insert([
    MyRow(key="1", name="Alice", age=30),
    MyRow(key="2", name="Bob", age=25),
    MyRow(key="1", name="Alice", age=30)
])

rows = tb.select(condition={"name": "Alice"})
print(rows)

row = tb.select_one(condition={"name": "Bob"})
print(row)

tb.insert_one({"key": "4", "name": "James", "age": 22})
tb.insert_one(MyRow(key="5", name="John", age=24))

rows = tb.select_all()
print(rows)

[Row(key='1', name='Alice', age=30), Row(key='1', name='Alice', age=30)]
Row(key='2', name='Bob', age=25)
[Row(key='5', name='John', age=24), Row(key='4', name='James', age=22), Row(key='1', name='Alice', age=30), Row(key='2', name='Bob', age=25), Row(key='1', name='Alice', age=30)]


In [5]:
# DB 스냅샷 테스트.
dbm.take_db_snapshot(name=db_name)

In [6]:
# 데이터 삭제 테스트.
tb.delete(condition={"age": 25})
rows = tb.select_all()
print(rows)

# 테이블 삭제 테스트.
db.drop_table(name=tb_new_name)
db.show_tables()

[Row(key='5', name='John', age=24), Row(key='4', name='James', age=22), Row(key='1', name='Alice', age=30), Row(key='1', name='Alice', age=30)]
- No table exists.


In [7]:
# 중복키 데이터 삽입 테스트.
db.create_table(name=tb_new_name, row_class=MyRow, primary_keys=["key", "name"])
tb = db.load_table(name=tb_new_name)
tb.insert([MyRow(key="1", name="Alice", age=30)])
try:
    tb.insert([MyRow(key="1", name="Alice", age=40)])
    db.drop_table(name=tb_new_name)
except Exception as e:
    print(e)

Table 'my_new_table' created successfully.
'Some data have duplicate keys.'


In [8]:
# DB 삭제 테스트.
dbm.drop_db(name=db_name)

In [9]:
# 휴지통 및 스냅샷 삭제 테스트.
dbm.clear_trash()
dbm.clear_snapshot()

dbm.show_dbs()

- Deleted 2 folders and 4 files from 'C:\Users\junhe\Documents\Python Scripts\bingle\test\file_db_data\.trash'.
- Deleted 1 folders and 2 files from 'C:\Users\junhe\Documents\Python Scripts\bingle\test\file_db_data\.snapshot'.
- No database exists.
